In [1]:
''' Setup '''
import os, sys,time
import pandas as pd
import numpy as np
from psychopy import prefs
prefs.hardware['audioLib'] = ['PTB']
from psychopy import sound
from psychopy import visual, monitors, gui, event, core, logging
import psychtoolbox as ptb 


from string import ascii_letters, digits
import numpy.matlib
import random
import math 
import matplotlib.pyplot as plt

# set to 1 if True
islaptop = 0
isfullscreen = 1
iseyetracking = 0
refreshrate = 60

if iseyetracking:
    import pylink
    from EyeLinkCoreGraphicsPsychoPy import EyeLinkCoreGraphicsPsychoPy

# Set up a  data folder to save the results
results_folder = 'data_temporal_estimation'
if not os.path.exists(results_folder):
    os.makedirs(results_folder)

# Prompt user to specify a filename
dlg_title = 'Enter Participant'
dlg_prompt = 'Please enter participant number'
fname = 'TEST'
while True:
    dlg = gui.Dlg(dlg_title)
    dlg.addText(dlg_prompt)
    dlg.addField('Participant Number:', fname)
    # show dialog and wait for OK or Cancel
    ok_data = dlg.show()
    if dlg.OK:  # if ok_data is not None
        print('Participant: {}'.format(ok_data[0]))
    else:
        print('user cancelled')
        core.quit()

    # get the string entered
    fname = 'P' + dlg.data[0]
    assert not os.path.exists('./' + results_folder +'/' + fname + '_trialmat.csv'), "FILE ALREADY EXISTS"

    # check if the filename is valid for eye tracking (length <= 8 & no special char)
    allowed_char = ascii_letters + digits + '_'
    if not all([c in allowed_char for c in fname]):
        print('ERROR: Invalid EDF filename')
    elif len(fname) > 8:
        print('ERROR: EDF filename should not exceed 8 characters')
    else:
        break
# fname = 'TEST_s'

def screeninfo(islaptop):
    if islaptop==1:
        monitorwidth = 33. # monitor width in cm
        viewdist = 50
    else:
        monitorwidth = 53.5 # monitor width in cm (64 for eyetracking pc, 60 for my desktop)
        viewdist = 57
    return {'viewdist':viewdist,'monitorwidth':monitorwidth}

def deg_to_pix(dva,screen_info,scn_width):
    size_cm = screen_info['viewdist']*np.tan(np.deg2rad(dva/2))*2
    pix_per_cm = scn_width/screen_info['monitorwidth']
    size_pix = size_cm*pix_per_cm
    return int(np.round(size_pix))

def pix_to_deg(full_size_pix,screen_info,scn_width):
    pix_per_cm = scn_width/screen_info['monitorwidth']
    size_cm = full_size_pix/pix_per_cm
    dva = (size_cm/2/screen_info['viewdist'])*2
    return np.rad2deg(dva)

def eyetracking_calib(win,el_tracker): 
    win.flip()
    try:
        el_tracker.doTrackerSetup()
    except RuntimeError as err:
        print('ERROR:', err)
        el_tracker.exitCalibration()
    el_tracker = pylink.getEYELINK()
    el_tracker.setOfflineMode()
    el_tracker.sendCommand('clear_screen 0')
    el_tracker.startRecording(1, 1, 1, 1)
    core.wait(1)

def eyetracker_exit(el_tracker,fname,results_folder):
    el_tracker.stopRecording()
    el_tracker.setOfflineMode()
    el_tracker.sendCommand('clear_screen 0')
    pylink.msecDelay(500)
    el_tracker.closeDataFile()
    print('EDF data is transferring from EyeLink Host PC...')
    el_tracker.receiveDataFile(fname + '.edf', results_folder +'/' + fname +'.edf')
    el_tracker.close()


pygame 1.9.6
Hello from the pygame community. https://www.pygame.org/contribute.html
Participant: 12012


In [2]:
# SET UP SCREEN ENVIRONMENT WITH AND WITHOUT EYETRACKER
win=visual.Window(color='Gray',colorSpace='rgb',units='pix',checkTiming=True,fullscr=isfullscreen,winType='pyglet',screen=0)
scn_width, scn_height = win.size
screen_info = screeninfo(islaptop=islaptop)

if iseyetracking:
    # Step 1: Connect to the EyeLink Host PC
    pylink.closeGraphics()
    try:
        el_tracker = pylink.EyeLink("100.1.1.1")
    except RuntimeError as error:
        print('ERROR:', error)
        core.quit()

    # Step 2: Open an EDF data file on the Host PC
    try:
        el_tracker.openDataFile(fname)
    except RuntimeError as err:
        print('ERROR:', err)
        if el_tracker.isConnected():
            el_tracker.close()
        core.quit()

    # Step 3: Configure the tracker
    el_tracker.setOfflineMode()
    vstr = el_tracker.getTrackerVersionString()
    eyelink_ver = int(vstr.split()[-1].split('.')[0])
    print('Running experiment on %s, version %d' % (vstr, eyelink_ver))
    file_event_flags = 'LEFT,RIGHT,FIXATION,SACCADE,BLINK,MESSAGE,BUTTON,INPUT'
    link_event_flags = 'LEFT,RIGHT,FIXATION,SACCADE,BLINK,BUTTON,FIXUPDATE,INPUT'
    if eyelink_ver > 3:
        file_sample_flags = 'LEFT,RIGHT,GAZE,HREF,RAW,AREA,HTARGET,GAZERES,BUTTON,STATUS,INPUT'
        link_sample_flags = 'LEFT,RIGHT,GAZE,GAZERES,AREA,HTARGET,STATUS,INPUT'
    else:
        file_sample_flags = 'LEFT,RIGHT,GAZE,HREF,RAW,AREA,GAZERES,BUTTON,STATUS,INPUT'
        link_sample_flags = 'LEFT,RIGHT,GAZE,GAZERES,AREA,STATUS,INPUT'
    el_tracker.sendCommand("file_event_filter = %s" % file_event_flags)
    el_tracker.sendCommand("file_sample_data = %s" % file_sample_flags)
    el_tracker.sendCommand("link_event_filter = %s" % link_event_flags)
    el_tracker.sendCommand("link_sample_data = %s" % link_sample_flags)

    # Choose a calibration type, H3, HV3, HV5, HV13 (HV = horizontal/vertical),
    el_tracker.sendCommand("calibration_type = HV5")
   
    # Step 4: set up a graphics environment for calibration
    el_coords = "screen_pixel_coords = 0 0 %d %d" % (scn_width - 1, scn_height - 1)
    el_tracker.sendCommand(el_coords)
    dv_coords = "DISPLAY_COORDS  0 0 %d %d" % (scn_width - 1, scn_height - 1)
    el_tracker.sendMessage(dv_coords)
    genv = EyeLinkCoreGraphicsPsychoPy(el_tracker, win)
    genv.setCalibrationColors((-1, -1, -1), win.color)

    # Use a picture as the calibration target (circle is default)
    genv.setTargetType('spiral')
    
    pylink.openGraphicsEx(genv)
    try:
        el_tracker.doTrackerSetup()
    except RuntimeError as err:
        print('ERROR:', err)
        el_tracker.exitCalibration()

In [3]:
# parameters
n_blocks = 4
total_time = 2.5 # time it would take to move all the way from start to finish
n_frames = total_time/(1/refreshrate)+1
assert n_frames - int(n_frames) == 0, 'total time has to be divisible by refresh rate'
fix_time = 1
fix_frames = int(fix_time/(1/refreshrate))
response_keys = ['1','0','q','s']

# sizes
fix_size = deg_to_pix(0.5,screen_info,scn_width)
box_pos = deg_to_pix(5,screen_info,scn_width)
box_pos=[-box_pos,-box_pos,box_pos,box_pos]

dist_start_end = deg_to_pix(13,screen_info,scn_width)
dist_start_end = dist_start_end - (dist_start_end % ((n_frames-1)/2)) # find a value that actually fits the total number of frames
start_pos = -dist_start_end
end_pos = dist_start_end
position_vector = np.linspace(start_pos,end_pos,int(n_frames))

distance_travelled_pix = end_pos-start_pos
distance_travelled_deg = pix_to_deg(distance_travelled_pix,screen_info,scn_width)
speed_per_sec=distance_travelled_deg/total_time

occluder_width_deg = 9
stim_size_deg=0.5
stim_size = deg_to_pix(stim_size_deg,screen_info,scn_width)
occluder_width = deg_to_pix(occluder_width_deg,screen_info,scn_width)

# calculate stop positions -- this is not relevant for this experiment but we need to do it so it's consistent with the speed task
n_stops = 7
min_max_stop_pos = (occluder_width/2)-(stim_size*2)

tmp = np.abs(position_vector - min_max_stop_pos)
max_stop_pos = np.max(np.where(np.min(tmp)==tmp))
tmp = np.abs(position_vector - -min_max_stop_pos)
min_stop_pos = np.min(np.where(np.min(tmp)==tmp))

stop_pos = np.round(np.linspace(min_stop_pos,max_stop_pos,n_stops))
all_stop_pos=[position_vector[int(i)] for i in stop_pos]
all_stop_pos

# reappearnace 
reappearance_loc = (occluder_width/2)-(stim_size/2)


In [4]:
# add new speeds, make sure that the stop positions are the same across 
from functools import reduce
def factors(n):    
    return (reduce(list.__add__, 
                ([i, n//i] for i in range(1, int(n**0.5) + 1) if n % i == 0)))

movement_resolution = np.unique(np.diff(position_vector))[0] # how many pixels between each position
num_frames_stop_pos = np.where(all_stop_pos[1]==position_vector)[0][0]-np.where(all_stop_pos[0]==position_vector)[0][0] # how many refreshs between stop positions
pix_between_stop_pos = movement_resolution*num_frames_stop_pos # how many pixels between stop positions

# we can only scale the speed in a certain way given that we want to make sure the stop positions stay consistent        
possible_factors=np.sort(factors(pix_between_stop_pos))
factors_slow = possible_factors[possible_factors>np.max([movement_resolution,num_frames_stop_pos])]
factors_fast = possible_factors[possible_factors<np.min([movement_resolution,num_frames_stop_pos])]

# adding 4 speeds in total, so we'll end with 5 (with the inital one being in the middle)
# pix_per_refresh = np.flip(np.sort(np.hstack((factors_slow[0:2],movement_resolution,factors_fast[-2:]))))
pix_per_refresh = np.flip(np.sort(np.hstack((factors_slow[0:1],movement_resolution,factors_fast[-1:]))))

pix_per_refresh = np.flip(np.arange(pix_per_refresh[-1],pix_per_refresh[0]))


all_pos = []
for i in pix_per_refresh:
    tmp = np.arange(0,end_pos+i,i)
    new_end_pos = tmp[np.argmin(end_pos-tmp)]
    all_pos.append(np.arange(new_end_pos*-1,new_end_pos+1,i))





In [5]:
# TRIALMATRIX
# So these speeds need to be defined in a way that we know for *certain* that the stop positions are in it 
n_directions = 2
n_unique_trials = 1
trl_cols = ['participant','block','trial','direction','start_pos','start_pos_deg','t_start','t_end','speed_idx','total_time','trial_time','reappearance_time','reappearance_loc','reappearance_loc_deg','pix_per_refresh','deg_per_sec','pix_per_sec','pos_pressed','pos_pressed_deg','error','error_deg','rt']
trialmat = pd.DataFrame(np.zeros((int(n_unique_trials),len(trl_cols))),columns=trl_cols)
trialmat['participant'] = fname
trialmat['block'] = 1
trialmat['trial'] = range(1,int(n_unique_trials)+1)
trialmat['direction']=1
trialmat['reappearance_loc'] = reappearance_loc
trialmat['reappearance_loc_deg'] = pix_to_deg(reappearance_loc,screen_info,scn_width)
trialmat['widthpix']=scn_width
trialmat['heightpix']=scn_height
trialmat['viewdist']=screen_info['viewdist']
trialmat['monitorwidth']=screen_info['monitorwidth']


# copy for different speeds
trialmats_tmp = []
for i in range(len(pix_per_refresh)):
    trialmats_a = trialmat.copy()
    trialmats_a.pix_per_refresh = pix_per_refresh[i]
    
    trialmats_a.start_pos = all_pos[i][0]
    trialmats_a.start_pos_deg = pix_to_deg(all_pos[i][0],screen_info,scn_width)

    trialmats_a.trial_time = np.abs(((trialmats_a.reappearance_loc-trialmats_a.start_pos)/trialmats_a.pix_per_refresh)*(1/refreshrate))
    trialmats_a.total_time = ((np.abs(trialmats_a.start_pos)*2)/trialmats_a.pix_per_refresh)*(1/refreshrate)
   
    trialmats_a.speed_idx = i
    distance_travelled_pix = all_pos[i][-1]-all_pos[i][0]
    distance_travelled_deg = pix_to_deg(distance_travelled_pix,screen_info,scn_width)
    trialmats_a.deg_per_sec=distance_travelled_deg/trialmats_a.total_time
    trialmats_a.pix_per_sec=distance_travelled_pix/trialmats_a.total_time

    # calculate when and where the object would re-appear if it just continued moving
    trialmats_a.reappearance_loc = reappearance_loc
    time_per_pixel = 1/trialmats_a.pix_per_sec
    trialmats_a.reappearance_time = np.abs(time_per_pixel*(trialmats_a.reappearance_loc-trialmats_a.start_pos))

    trialmats_tmp.append(trialmats_a)
trialmats = pd.concat(trialmats_tmp)


# reverse for different directions
trialmat_r = trialmats.copy()
trialmat_r['start_pos'] = trialmat_r['start_pos']*-1
trialmat_r['start_pos_deg'] = trialmat_r['start_pos_deg']*-1
trialmat_r['direction']=2
trialmat_r.reappearance_loc = -reappearance_loc
time_per_pixel = 1/trialmat_r.pix_per_sec
trialmat_r.reappearance_time = np.abs(time_per_pixel*(trialmat_r.reappearance_loc-trialmat_r.start_pos))
trialmat_r.trial_time = np.abs(((trialmat_r.reappearance_loc-trialmat_r.start_pos)/trialmat_r.pix_per_refresh)*(1/refreshrate))

trialmats = pd.concat([trialmats,trialmat_r],ignore_index=True)


# randomize and stack some trialmats to generate multiple blocks
trialmats=trialmats.sample(frac=1)
trialmats.reset_index(drop=True,inplace=True)
for b in range(n_blocks):
    tmp=trialmats.sample(frac=1)
    tmp.reset_index(drop=True,inplace=True)
    tmp['block'] = b+1

    if b ==0:
        trial_df = tmp
    else: 
        trial_df = pd.concat([trial_df,tmp],ignore_index=True)
    

trial_df.columns

Index([&#39;participant&#39;, &#39;block&#39;, &#39;trial&#39;, &#39;direction&#39;, &#39;start_pos&#39;,
       &#39;start_pos_deg&#39;, &#39;t_start&#39;, &#39;t_end&#39;, &#39;speed_idx&#39;, &#39;total_time&#39;,
       &#39;trial_time&#39;, &#39;reappearance_time&#39;, &#39;reappearance_loc&#39;,
       &#39;reappearance_loc_deg&#39;, &#39;pix_per_refresh&#39;, &#39;deg_per_sec&#39;, &#39;pix_per_sec&#39;,
       &#39;pos_pressed&#39;, &#39;pos_pressed_deg&#39;, &#39;error&#39;, &#39;error_deg&#39;, &#39;rt&#39;,
       &#39;widthpix&#39;, &#39;heightpix&#39;, &#39;viewdist&#39;, &#39;monitorwidth&#39;],
      dtype=&#39;object&#39;)

In [6]:
# visual objects
occluder = visual.Rect(win=win, width=occluder_width, height=int(occluder_width/4), units='pix', lineWidth=2, lineColor=None,fillColor='DimGrey', pos=[0,0],opacity=1)
fixation = []
fixation.append(visual.Circle(win=win,size = fix_size,fillColor='grey',units='pix',lineWidth=0))
fixation.append(visual.Rect(win=win,width = fix_size/8,height=fix_size,fillColor='grey',lineWidth=0))
fixation.append(visual.Rect(win=win,height = fix_size/8,width=fix_size,fillColor='grey',lineWidth=0))
stim = visual.circle.Circle(win, size = stim_size, units='pix', lineWidth=0, fillColor='orange', pos=(0, 0))
marker = visual.Polygon(win=win,fillColor='black',size = fix_size,edges=3)

# sound object
mysound = sound.Sound('C',secs=20, hamming=False, syncToWin=True)


In [7]:
calib_next_trial,block_end = 1,1
for t in range(len(trial_df)):
    if calib_next_trial:
        pressed=[]
        while 1:  
            if block_end==1:
                text=visual.TextStim(win,text='Block ' + str(trial_df.loc[t,'block']) + ' out of ' +str(n_blocks) +'\n\n Press SPACE to start.',height=40)
            else:
                text = visual.TextStim(win,text='Take a short break!\n\n Press SPACE to continue.',height=40)
            text.draw()
            win.flip()
            pressed=event.getKeys(keyList='space', modifiers=False, timeStamped=False) 
            if pressed:
                break
        # eyetracking calibration
        if iseyetracking: 
            text=visual.TextStim(win,text='Eye-tracker (re-)calibration...',height=30)
            text.draw()
            win.flip()
            core.wait(2)
            eyetracking_calib(win,el_tracker)

    win.mouseVisible = False
    # draw fixation
    for f in range(fix_frames):
        for i in fixation:  
            i.draw()
        win.flip()

    # draw stimuli and play sound
    if iseyetracking:
        el_tracker.sendMessage('TRIAL %d stim_on' % t)
        el_tracker.sendCommand("record_status_message '%s'" % 'TRIAL number %d stim_on' % t)

    next_flip = win.getFutureFlipTime(clock='ptb')
    mysound.play(when=next_flip)
    if trial_df.loc[t,'direction']==1:
        pos_vec = all_pos[trial_df.loc[t,'speed_idx']]
        pos_vec = pos_vec[pos_vec<reappearance_loc]
    else:
        pos_vec = np.flip(all_pos[trial_df.loc[t,'speed_idx']])
        pos_vec = pos_vec[pos_vec>-reappearance_loc]

    # wait for response and end trial when participant has pressed the button
    resp = ()
    for i,j in enumerate(pos_vec):
        stim.pos = (j,0)
        stim.draw()
        occluder.draw()
        if i == 0:
            trial_df.loc[t,'t_start'] = win.flip()
        else:
            trial_df.loc[t,'t_end']=win.flip()
        resp=event.getKeys(keyList='space', modifiers=False, timeStamped=True)
        if any(resp):
            mysound.stop()
            if iseyetracking:
                el_tracker.sendMessage('TRIAL %d stim_off' % t)
                el_tracker.sendCommand("record_status_message '%s'" % 'TRIAL number %d stim_off' % t)
            break

    # wait for the participant to press button or until 2 s have passed
    if not any(resp):
        t1 = time.time()
        while 1:
            stim.draw()
            occluder.draw()
            trial_df.loc[t,'t_end']=win.flip()
            resp=event.getKeys(keyList='space', modifiers=False, timeStamped=True)
            if any(resp) or (time.time()-t1)>2:
                mysound.stop()
                break

    if any(resp):
        trial_df.loc[t,'rt'] = resp[0][1]-trial_df.loc[t,'t_start']
        if trial_df.loc[t,'direction']==1:
            trial_df.loc[t,'pos_pressed']=trial_df.loc[t,'start_pos']+trial_df.loc[t,'rt']*trial_df.loc[t,'pix_per_sec']
            trial_df.loc[t,'error']=(trial_df.loc[t,'reappearance_loc']-trial_df.loc[t,'pos_pressed'])
        else:
            trial_df.loc[t,'pos_pressed']=trial_df.loc[t,'start_pos']-trial_df.loc[t,'rt']*trial_df.loc[t,'pix_per_sec']
            trial_df.loc[t,'error']=(trial_df.loc[t,'reappearance_loc']-trial_df.loc[t,'pos_pressed'])*-1
        trial_df.loc[t,'error_sec']=trial_df.loc[t,'rt']-trial_df.loc[t,'reappearance_time']
        trial_df.loc[t,'error_deg'] = pix_to_deg(trial_df.loc[t,'error'],screen_info,scn_width) 
        trial_df.loc[t,'pos_pressed_deg'] = pix_to_deg(trial_df.loc[t,'pos_pressed'],screen_info,scn_width) 

    # add 1 sec blank screen
    win.flip()
    core.wait(1)

    # show a bit of text before next trial
    txt = visual.TextStim(win,text='Get ready...',pos=(0,0),height=30)
    t1 = time.time()
    while 1:
        txt.draw()
        win.flip()
        pressedq=event.getKeys(keyList=['q'], modifiers=False, timeStamped=False) 
        if pressedq:
            win.close()
        if (time.time()-t1)>1:
            break

    if np.any(np.isin(np.where(np.diff(trial_df['block']))[0],t)):
        calib_next_trial = 1
        block_end = 1
    else: 
        calib_next_trial = 0
        block_end = 0


# End of the experiment -- save data and close eyetracker
txt=visual.TextStim(win=win,text='End of experiment :)',height = 30)
txt.draw()
win.flip()
core.wait(5)
win.close()
trial_df.to_csv('./' + results_folder +'/' + fname + '_trialmat.csv')

if iseyetracking:
    try:
        eyetracker_exit(el_tracker,fname,results_folder)
    except RuntimeError as error:
        print('that did not work...')


5.7589 	WARNING 	Monitor specification not found. Creating a temporary one...
5.7610 	WARNING 	User requested fullscreen with size [800 600], but screen is actually [1920, 1080]. Using actual size
